# ANT-X: AI Augmented Networking Tool for Transportation Analysis, Modeling, and Simulation

<div style="display: flex; justify-content: space-around; align-items: center;">
    <img src="elements/logos/logo_12.png" alt="Image 1" style="width:33%;">
    <img src="elements/logos/logo_16.png" alt="Image 2" style="width:33%;">
    <img src="elements/logos/logo_18.png" alt="Image 2" style="width:33%;">
</div>

## Introduction
The dependensies of ANT-X is

In this notebook, we will record the framework and almost every designed details of the ANT-X. ANT-X look forward to

In [1]:
import pandas as pd

In [1]:
from dash import Dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd

# Initialize Dash app with suppressed callback exceptions
app = Dash(__name__, suppress_callback_exceptions=True)

# ==================== Macro-level State Network ====================
macro_edges = pd.DataFrame({'source': ['A', 'B'], 'destination': ['B', 'C']})
macro_nodes = ['A', 'B', 'C']
macro_pos = {'A': (0, 1), 'B': (1, 0), 'C': (2, 1)}

def create_macro_fig(highlight_node=None, highlight_link=None):
    fig = go.Figure()
    for _, row in macro_edges.iterrows():
        x0, y0 = macro_pos[row['source']]
        x1, y1 = macro_pos[row['destination']]
        edge_name = f"Edge: {row['source']} -> {row['destination']}"
        is_highlighted = highlight_link == edge_name
        fig.add_trace(go.Scatter(
            x=[x0, x1], y=[y0, y1], mode='lines',
            line=dict(width=6 if is_highlighted else 2, color='orange' if is_highlighted else 'lightgray'),
            hoverinfo='text', text=edge_name, name=edge_name
        ))
    for node, (x, y) in macro_pos.items():
        is_highlighted = highlight_node == node
        fig.add_trace(go.Scatter(
            x=[x], y=[y], mode='markers+text',
            marker=dict(size=25 if is_highlighted else 15, color='red' if is_highlighted else 'lightgray'),
            text=node, textposition='top center', hoverinfo='text', name=f"Node: {node}"
        ))
    fig.update_layout(
        title="Macro-Level State Network", showlegend=False,
        clickmode='event+select', hovermode='closest'
    )
    return fig

macro_fig = create_macro_fig()

# ==================== Micro-level Spatiotemporal Network ====================
micro_edges = pd.DataFrame({'source': ['M1', 'M2', 'M3'], 'destination': ['M2', 'M3', 'M4']})
micro_nodes = ['M1', 'M2', 'M3', 'M4']
micro_pos = {'M1': (0, 0), 'M2': (1, 1), 'M3': (2, 1), 'M4': (3, 0)}

def create_micro_fig(highlight_nodes=None, highlight_edges=None):
    highlight_nodes = highlight_nodes or []
    highlight_edges = highlight_edges or []
    fig = go.Figure()
    for _, row in micro_edges.iterrows():
        x0, y0 = micro_pos[row['source']]
        x1, y1 = micro_pos[row['destination']]
        edge_name = f"Edge: {row['source']} -> {row['destination']}"
        is_highlighted = edge_name in highlight_edges
        fig.add_trace(go.Scatter(
            x=[x0, x1], y=[y0, y1], mode='lines',
            line=dict(width=6 if is_highlighted else 2, color='blue' if is_highlighted else 'lightgray'),
            hoverinfo='text', text=edge_name, name=edge_name
        ))
    for node, (x, y) in micro_pos.items():
        is_highlighted = node in highlight_nodes
        fig.add_trace(go.Scatter(
            x=[x], y=[y], mode='markers+text',
            marker=dict(size=20 if is_highlighted else 10, color='green' if is_highlighted else 'gray'),
            text=node, textposition='top center', hoverinfo='text', name=f"Node: {node}"
        ))
    fig.update_layout(title="Micro-Level Spatiotemporal Network", showlegend=False)
    return fig

micro_fig = create_micro_fig()

# ==================== Mapping Logic ====================
macro_to_micro_node_map = {'A': 'M1', 'B': 'M2', 'C': 'M3'}
macro_to_micro_edge_map = {
    'Edge: A -> B': ['Edge: M1 -> M2'],  # Macro edge maps to a single micro edge
    'Edge: B -> C': ['Edge: M2 -> M3', 'Edge: M3 -> M4']  # Macro edge maps to multiple micro edges
}

# ==================== Layout ====================
app.layout = html.Div([
    html.H3("Two-Layer Network Interaction System"),
    dcc.Store(id='highlight-state'),
    html.Label("Select Interaction Mode:"),
    dcc.Dropdown(
        id='interaction-mode',
        options=[{'label': 'Select Node', 'value': 'node'}, {'label': 'Select Link', 'value': 'link'}],
        value='node'
    ),
    html.Div(id='selection-panel', style={'margin-top': '10px'}),
    html.Button("Select", id='select-button', n_clicks=0, style={'margin-top': '10px'}),
    html.Div([
        dcc.Graph(id='macro-network', figure=macro_fig, config={'displayModeBar': False}, style={'flex': '50%'}),
        dcc.Graph(id='micro-network', figure=micro_fig, config={'displayModeBar': False}, style={'flex': '50%'})
    ], style={'display': 'flex'}),
    html.Div(id='debug-output', style={'margin-top': '20px', 'font-size': '18px', 'color': 'blue'}),
])

# ==================== Callbacks ====================
@app.callback(
    Output('selection-panel', 'children'),
    Input('interaction-mode', 'value')
)
def show_selection_panel(interaction_mode):
    if interaction_mode == 'node':
        return html.Div([
            html.Label("Select Node:"),
            dcc.Dropdown(
                id='node-selection',
                options=[{'label': node, 'value': node} for node in macro_nodes],
                placeholder="Select a node"
            )
        ])
    elif interaction_mode == 'link':
        return html.Div([
            html.Label("Select Link:"),
            dcc.Dropdown(
                id='link-source',
                options=[{'label': node, 'value': node} for node in macro_nodes],
                placeholder="Select source node"
            ),
            dcc.Dropdown(
                id='link-destination',
                options=[{'label': node, 'value': node} for node in macro_nodes],
                placeholder="Select destination node",
                style={'margin-top': '10px'}
            )
        ])

@app.callback(
    [Output('highlight-state', 'data'), Output('debug-output', 'children')],
    Input('select-button', 'n_clicks'),
    [State('interaction-mode', 'value'), State('selection-panel', 'children')]
)
def update_highlight_state(n_clicks, interaction_mode, panel_children):
    if n_clicks == 0:
        return None, html.Div("No selection made yet.")

    def get_dropdown_value(panel, component_id):
        if not panel or 'props' not in panel or 'children' not in panel['props']:
            return None
        for child in panel['props']['children']:
            if isinstance(child, dict) and 'props' in child and child['props'].get('id') == component_id:
                return child['props'].get('value')
        return None

    if interaction_mode == 'node':
        node_selection = get_dropdown_value(panel_children, 'node-selection')
        if node_selection:
            return {'node': node_selection, 'link': None}, html.Div(f"Node selected: {node_selection}")

    if interaction_mode == 'link':
        link_source = get_dropdown_value(panel_children, 'link-source')
        link_destination = get_dropdown_value(panel_children, 'link-destination')
        if link_source and link_destination:
            selected_link = f"Edge: {link_source} -> {link_destination}"
            return {'node': None, 'link': selected_link}, html.Div(f"Link selected: {selected_link}")

    return None, html.Div("No valid selection.")

@app.callback(
    [Output('macro-network', 'figure'), Output('micro-network', 'figure')],
    Input('highlight-state', 'data')
)
def update_networks(highlight_state):
    if not highlight_state:
        return create_macro_fig(), create_micro_fig()

    highlight_node = highlight_state.get('node')
    highlight_link = highlight_state.get('link')

    micro_highlight_nodes = [macro_to_micro_node_map[highlight_node]] if highlight_node in macro_to_micro_node_map else []
    micro_highlight_edges = macro_to_micro_edge_map.get(highlight_link, [])

    return create_macro_fig(highlight_node=highlight_node, highlight_link=highlight_link), \
           create_micro_fig(highlight_nodes=micro_highlight_nodes, highlight_edges=micro_highlight_edges)

# ==================== Run the App ====================
if __name__ == '__main__':
    app.run_server(debug=True)